In [1]:
import os
from IPMOFresults import *

# Folder containing results file
#outputDir = "C:\\Kutay\\IPMOF_Output"
outputDir = r'C:\Kutay\IPMOF_Experimental\Results_PBC'

In [2]:
baseMOF = os.listdir(outputDir)
table = []
totalStructureCount = 0
completed = '---'
simCount = 0
IPMOFs = []
minStructures = 2
strIdx = 0

for bMOF in baseMOF:
    mobileMOF = os.listdir(os.path.join(outputDir,bMOF))
    
    for mMOF in mobileMOF:
        simPar = os.listdir(os.path.join(outputDir,bMOF, mMOF))
        
        for sim in simPar:
            simDir = os.path.join(outputDir, bMOF, mMOF, sim)
            resultsDir = os.path.join(outputDir,bMOF, mMOF, sim, 'results.txt')
            
            # Check if job .out file exists
            for simFile in os.listdir(simDir):
                if '.out' in simFile:
                    jobFileDir = os.path.join(outputDir,bMOF, mMOF, sim, simFile)
                    runTime = readJobFile(jobFileDir)
                    #runTime = '01:00'
                    break
                else:
                    runTime = '---'
            
            # Check if results file exists
            if os.path.isfile(resultsDir):
                resultsSize = os.path.getsize(resultsDir)
                
                if resultsSize > 0:
                    summary = readSummary(simDir)
                    IPstructures = readStructures(simDir)
                    readUnitCell(IPstructures, simDir)
                    if len(IPstructures['xyz']) > 0:
                        minEstructures = getMinEnergyStructures(IPstructures, minStructures)
                        IPMOFs.append([])
                        for minStrIdx, minStr in enumerate(minEstructures):
                            IPMOFs[strIdx].append({})
                            IPMOFs[strIdx][minStrIdx]['xyz'] = IPstructures['xyz'][minStr]
                            IPMOFs[strIdx][minStrIdx]['numAtoms'] = IPstructures['numAtoms'][minStr]
                            IPMOFs[strIdx][minStrIdx]['name'] = bMOF + '_' + mMOF
                            IPMOFs[strIdx][minStrIdx]['bMOF_UCangle'] = IPstructures['bMOF_UCangle']
                            IPMOFs[strIdx][minStrIdx]['bMOF_UCsize'] = IPstructures['bMOF_UCsize']
                            IPMOFs[strIdx][minStrIdx]['mMOF_UCangle'] = IPstructures['mMOF_UCangle']
                            IPMOFs[strIdx][minStrIdx]['mMOF_UCsize'] = IPstructures['mMOF_UCsize']
                        strIdx += 1
                    if len(summary['percent']) > 0:
                        completed = summary['percent'][-1]
                        totalStructureCount = summary['structureCount'][-1]
                    else:
                        completed = 'Interrupted'
                        totalStructureCount = 'Interrupted'
                        totalStructureCount = IPstructures['structureIndex'][-1]
                else:
                    completed = 'Error'
                    totalStructureCount = 'Err'
                    
            else:
                completed = 'NoResults'
                totalStructureCount = 'NoResults'
                           
        table.append([])
        table[simCount].append(bMOF)
        table[simCount].append(mMOF)
        table[simCount].append(sim)
        table[simCount].append(completed)
        table[simCount].append(runTime)
        table[simCount].append(totalStructureCount)
        simCount += 1

completeSIM = []
errorSIM = []
interruptedSIM = []
noresSIM = []
for refSim in table:
    if refSim[3] == 100:
        completeSIM.append(refSim)
    if refSim[3] == 'Error':
        errorSIM.append(refSim)
    if refSim[3] == 'Interrupted':
        interruptedSIM.append(refSim)
    if refSim[3] == 'NoResults':
        noresSIM.append(refSim)

del table

from tabulate import tabulate

headers = ["Base MOF", "Mobile MOF", "SimPar", "Completed", "RunTime", "sCount"]
print(tabulate(completeSIM, headers))

simTime = []
for comSim in completeSIM:
    if comSim[4] > 0:
        simTime.append(comSim[4])
    else:
        # If simulation started before end of day and finished after midnight
        simTime.append(comSim[4]+24*3600)
        
# Index of simulation with maximum time
maxTimeIndex = simTime.index(sorted(simTime)[-1])
minTimeIndex = simTime.index(sorted(simTime)[0])
#print(completeSIM[maxTimeIndex])

# Convert time from seconds to string format
import datetime
print('Maximum time: ' + str(datetime.timedelta(seconds=simTime[maxTimeIndex])))
print('Minimum time: ' + str(datetime.timedelta(seconds=simTime[minTimeIndex])))

# Average Time
avgTime = sum(simTime)/len(simTime)
print('Average Time: ' + str(datetime.timedelta(seconds=avgTime)))

Base MOF    Mobile MOF    SimPar           Completed    RunTime    sCount
----------  ------------  -------------  -----------  ---------  --------
COGCOS-P1   COGCOS-P2     E-2_F12_L50_S          100         66         3
ETAXAS-P1   ETAXAS-P2     E-2_F12_L50_S          100         39         4
PELQII-P1   PELQII-P2     E-2_F12_L50_S          100        332         2
TIVYED-P1   TIVYED-P2     E-2_F12_L50_S          100        111         4
UNAZIT-P1   UNAZIT-P2     E-2_F12_L50_S          100        165         2
WEBZEK-P1   WEBZEK-P2     E-2_F12_L50_S          100        347        29
Maximum time: 0:05:47
Minimum time: 0:00:39
Average Time: 0:02:56.666667


In [3]:
exportDir = r'C:\Kutay\IPMOF_Experimental\Structures_PBC'
for IPMOF in IPMOFs:
    exportIPpdb(IPMOF[0], IPMOF[0]['name'], exportDir)

IndexError: list index out of range

In [4]:
IPMOFs[0][0]['xyz'][0].split()[3]

'10.23'

In [7]:
for i in range(1):
    print('asd \a asdsd')

asd  asdsd


In [8]:
IPstructures['xyz'][0][0]

'H 1.096 8.67 21.13'

In [5]:
for coorIndex, coor in enumerate(IPMOFs[0][0]['xyz']):
    coorLine = 'HETATM    ' + str(coorIndex) + ' ' + coor.split()[0] + '   UNK     1       '
    coorLine += coor.split()[1] + '   ' + coor.split()[2] + '   ' + coor.split()[3]
    coorLine += '  1.00  0.00           ' + coor.split()[0]

In [6]:
coorLine

'HETATM    646 H   UNK     1       11.048   1.428   8.67  1.00  0.00           H'

In [22]:
outputDir = r'C:\Kutay\IPMOF_MOF5\lowE2\MOF5\MOF5\simulPar'
resultsFileName = 'results.txt'
resDir = os.path.join(outputDir, resultsFileName)
resFile = open(resDir, 'r')
resLines = resFile.readlines()
resFile.close()

In [48]:

structureCount = 0

for line in resLines:
    if 'Base' in line:
        break
        
    if '---' in line:
        structureCount += 1
        lineIndex = resLines.index(line) + 1
        nextLine = resLines[lineIndex]
        if '---' in nextLine:
            structureCount -= 1
        if 'Base' in nextLine:
            structureCount -= 1


In [13]:
IPstructures['xyz'][0]

['H 1.096 8.67 21.13',
 'H 0.752 7.379 21.95',
 'H 0.362 8.79 22.51',
 'C 14.002 1.249 16.30',
 'C 13.956 0.448 15.05',
 'C 12.822 0.33 14.25',
 'H 12.059 0.825 14.45',
 'H 16.58 0.04 9.11',
 'C 14.002 5.057 16.84',
 'C 13.902 6.091 15.75',
 'C 15.111 6.29 14.97',
 'H 15.905 5.838 15.15',
 'C 14.994 7.206 13.94',
 'C 13.978 8.007 13.71',
 'C 12.856 7.863 14.5',
 'H 12.137 8.446 14.48',
 'C 12.82 6.82 15.57',
 'H 12.053 6.67 16.07',
 'C 13.967 9.085 12.67',
 'C 14.925 10.085 12.67',
 'H 15.559 10.083 13.35',
 'C 14.998 11.099 11.70',
 'H 15.686 11.724 11.70',
 'C 13.987 11.125 10.74',
 'C 12.982 10.132 10.73',
 'H 12.319 10.119 10.07',
 'C 13.026 9.194 11.71',
 'C 13.996 12.11 9.65',
 'C 17.289 6.705 13.30',
 'H 17.76 7.007 14.08',
 'H 17.859 6.78 12.53',
 'H 17.028 5.789 13.42',
 'C 10.638 8.262 11.0',
 'H 9.994 8.67 11.61',
 'H 10.338 7.379 10.79',
 'H 10.728 8.79 10.23',
 'Cd 13.961 2.918 18.56',
 'Cd 16.635 2.931 16.37',
 'O 12.842 1.858 16.61',
 'O 14.989 1.418 16.97',
 'O 15.022 4

In [13]:
IPstructures['xyz'][0][0]

'O 6.458 6.458 6.45'

In [19]:
exportDir = r'C:\Kutay\IPMOF_MOF5\lowE2\MOF5\MOF5\simulPar'
xyzFileName = 'MOF5IP'
IPindex = 0


In [20]:
exportIPxyz(0, xyzFileName, exportDir)

In [2]:
sourceDir = r'C:\Kutay\poreblazer_v3.0.2_d\Source'

In [5]:
os.listdir(sourceDir)

['defaults.dat', 'poreblazer.exe', 'UFF.atoms']

In [7]:
import shutil

In [9]:
shutil.copy?